### 1. This modolue is to figure out how to run the segmentation for the video and laod the packages

1. pip install stardist: https://pypi.org/project/stardist/
2. pip install tensorflow: https://www.tensorflow.org/install/pip
3. please be noted to install the specify tensorflow and pytorch version that work with the same compatible cuda
4. This code will take memory to save the segmented volumne for each frame as h5 -->

In [1]:
import sys
# from Segmentation import *
from Segmentation.Video_seg import *

2024-03-27 22:40:51.191259: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


True
Tesla P100-PCIE-12GB
True
2.13.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


#### 2. Load the model for 3D segmentation

In [2]:
model_weights_path = '/work/venkatachalamlab/Hang/GNN_matching_git/code/GNN_match/Segmentation/model_weights/weights_best_42stacks_all.h5'
model, model_2D = load_model_3D_and_2D(model_weights_path)

use_gpu:  True


base_model.py (149): output path for model already exists, files may be overwritten: /work/venkatachalamlab/Hang/GNN_matching_git/code/GNN_match/models/stardist
2024-03-27 22:41:07.918258: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11437 MB memory:  -> device: 0, name: Tesla P100-PCIE-12GB, pci bus id: 0000:03:00.0, compute capability: 6.0


Using default values: prob_thresh=0.5, nms_thresh=0.4.
There are 4 registered models for 'StarDist2D':

Name                  Alias(es)
────                  ─────────
'2D_versatile_fluo'   'Versatile (fluorescent nuclei)'
'2D_versatile_he'     'Versatile (H&E nuclei)'
'2D_paper_dsb2018'    'DSB 2018 (from StarDist 2D paper)'
'2D_demo'             None
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


#### 3. Run Segmentation for the video and Please specify the arguments: 

'load_path': the folder to load the original h5 file \
'save_folder: the folder to save segmentation file  \
'ch: the channel index used for tracking \
'zoom_factor': for low resolution zoom_factor = 2, staticfluild use zoom_factor = 1 \
'start_idx': the starting time index \
'end_idx': the ending time index \

In [3]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


Physical devices cannot be modified after being initialized


In [4]:
load_folder = '/work/venkatachalamlab/Hang/GNN_matching_git/dataset/ZM9624/'
save_folder = 'seg'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
    


In [9]:
img_shape = get_img_shape(load_folder)
[ch, zoom_factor] = [1,2]
for t_idx in tqdm(range(img_shape[0])):
# for t_idx in tqdm(range(2)):
    label_z = get_frame_segmentation(model,model_2D,t_idx, ch, zoom_factor,load_folder)
    # label[t_idx,0] = label_z
    save_var_h5('seg/'+str(t_idx)+'.h5',[label_z],['label'])
print("finish the segmentation of movie in the folder path: ", save_folder)

100%|██████████| 2/2 [00:43<00:00, 21.81s/it]

finish the segmentation of movie in the folder path:  seg


#### 4. The following version could run the above in parallel, needs to specify the memory

In [9]:
# if __name__ == "__main__":
#     # Define the range of iterations you want to process in parallel
#     img_shape = get_img_shape(load_folder)
#     [ch, zoom_factor] = [1,2] ## (for low resolution zoom_factor = 2, staticfluild use zoom_factor = 1)
#     start_idx = 0  ## starting time index
#     # end_idx = img_shape[0]   ## end time index
#     end_idx = 2   

#     # Create a ThreadPoolExecutor for parallel processing
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         # Use partial to create a function with fixed arguments (img_shape, model, model_2D, ch, zoom_factor, save_folder)
#         process_partial = partial(process_iteration,  model=model, model_2D=model_2D, ch=ch, zoom_factor=zoom_factor, 
#                                   load_folder=load_folder, save_folder=save_folder)

#         # Iterate and process in parallel using the ThreadPoolExecutor
#         tqdm_parallel = tqdm(range(start_idx, end_idx, 1))
#         # tqdm_parallel = tqdm(not_exits)
#         results = list(executor.map(process_partial, tqdm_parallel))

#     # Print the results (t_idx values)
#     print("Processed t_idx values:", results)